In [554]:
%pip install pandas

24619.44s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


## モデルの読み込み

In [555]:
from gensim.models import KeyedVectors

wv = KeyedVectors.load_word2vec_format('../create_model/wiki.model', binary=True)

## 類似趣味抽出

In [556]:
import pandas as pd

※ プロフィールは架空の人物です。
ChatGPTで生成しました

In [747]:
profile_df = pd.read_csv("./profile_multi_hobbies.csv")
profile_df

,id,name,hobby
0,A001001,松潤,旅行;サッカー;ゲーム;料理;映画鑑賞;アウトドア;散策
1,A000102,石原サトミ,ゴルフ;ヨガ;読書;散歩;ピクニック;体操
2,A002003,綾野剛,読書;散歩;料理;音楽;アウトドア;ガーデニング
3,A003004,綾瀬ハルカ,料理;映画鑑賞;音楽;アート;ダンス;ファッション
4,A004005,嵐オールスターズ,スポーツ観戦;散歩;バスケットボール;音楽;ゴルフ;カメラ
...,...,...,...
76,A076077,純子,ビーチバレー;ダンス;音楽;ピクニック;サッカー;アウトドア
77,A077078,勇太,ダンス;読書;バスケットボール;音楽;料理;カフェ巡り
78,A078079,真美,読書;音楽;ダンス;アート;映画鑑賞;博物館巡り
79,A079080,亜美,ランニング;ゴルフ;ピクニック;アート;音楽;サッカー


In [726]:
def similar_hobby_person(hobby, df):
    similar_keywords = wv.most_similar(positive=[hobby], topn=15)
        
    # # 指定された趣味に一致する人物のIDを取得
    # person_id_list = profile_df[profile_df['hobby'].str.contains(hobby)]['id']

    person_id_store = []
    similar_keywords.append((hobby,1.0))
    for similar_keyword in similar_keywords:
        keyword = similar_keyword[0]

        # 指定された趣味に一致する人物のIDを取得
        person_id_list = profile_df[profile_df['hobby'].str.contains(keyword)]['id']

        if not person_id_list.empty:
            for person_id in person_id_list:
                # 類似キーワードが既にあった場合は処理をやめる
                if person_id in person_id_store:
                    continue

                # 趣味が検索キーワードと同じ場合は類似度を1.0とする
                if hobby in profile_df[profile_df['id'] == person_id]['hobby'].values[0].split(';'):
                    cos_similarity = 1.0
                else:
                    cos_similarity = similar_keyword[1] # cos類似度取得

                if person_id in df['id'].values:
                    # 既に同じ ID が DataFrame に存在する場合、similarity リストに類似度を追加
                    index = df[df['id'] == person_id].index[0]
                    df.at[index, 'similarity'].append(cos_similarity)
                else:
                    # 新しい ID の場合、新しい行として DataFrame に追加
                    new_row = pd.DataFrame({'id': [person_id], 'similarity': [[cos_similarity]]})
                    df = pd.concat([df, new_row], ignore_index=True)
                person_id_store.append(person_id)
    return df

# ---------------------
# test
# ---------------------
# 既存のDataFrameを作成
df = pd.DataFrame(columns=['id', 'similarity'])

# 例: '旅行' 趣味と類似度 0.8 の人物を追加
df = similar_hobby_person('ビデオゲーム',df)
df = similar_hobby_person('サッカー', df)
df = similar_hobby_person('料理', df)
df

,id,similarity
0,A001001,"[0.6813110113143921, 1.0, 1.0]"
1,A016017,[1.0]
2,A044045,"[1.0, 1.0]"
3,A004005,[0.6596197485923767]
4,A019020,"[0.6596197485923767, 1.0]"
5,A075076,"[0.6596197485923767, 1.0]"
6,A005006,[0.6249154210090637]
7,A027028,[0.6249154210090637]
8,A028029,[0.6249154210090637]
9,A029030,[0.6249154210090637]


# 類似趣味検索
`search_target_parson_id`に任意のidを入れるとその人物の趣味と類似する人物の一覧を出力できます。

In [748]:
# ------------------------------------
# 検索したいユーザのIDを入力
search_target_person_id = "A062063"
# ------------------------------------

# DataFrameを作成
df = pd.DataFrame(columns=['id', 'similarity'])

# 検索対象の趣味を取得
search_target_person_record = profile_df[profile_df['id'] == search_target_person_id]
search_target_hobbies = search_target_person_record['hobby'].values[0]
search_target_hobby_list = search_target_hobbies.split(';')

search_target_hobby_list

# 類似度を計算するための既存のコード
for hobby in search_target_hobby_list:
    try:
        df = similar_hobby_person(hobby,df)
    except KeyError as e:
        Warning
        print('catch KeyError:',e)


# similarityカラム内のリストの平均を計算
df['similarity'] = df['similarity'].apply(lambda x: sum(x) / len(search_target_hobby_list) if len(x) > 0 else 0)


# 平均値に基づいてデータフレームをランキング
ranked_df = df.sort_values(by='similarity', ascending=False).reset_index(drop=True)

# ranked_dfからidとsimilarity列を取得
ranked_info = ranked_df[['id', 'similarity']]

# profile_dfからranked_infoに含まれるidの行を選択
selected_profiles = profile_df[profile_df['id'].isin(ranked_info['id'])]

selected_profiles
# ranked_infoとselected_profilesを結合

result_df = pd.merge(selected_profiles,ranked_info, on='id', how='right')

# 結果を表示
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print(f'検索ID: {search_target_person_id}')
print('================================検索対象プロフィール================================')
print(profile_df[profile_df['id'] == search_target_person_id])
print('---------------------------------------------------------------------------------')
result_df

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
検索ID: A062063
================================検索対象プロフィール================================
         id name                  hobby
62  A062063   健太  登山;読書;音楽;アート;ピクニック;料理
---------------------------------------------------------------------------------


,id,name,hobby,similarity
0,A062063,健太,登山;読書;音楽;アート;ピクニック;料理,1.000000
1,A024025,ゆうた,アート;ピクニック;釣り;ゴルフ;読書;料理,0.666667
2,A052053,真治,ビーチバレー;読書;ピクニック;アート;音楽;映画,0.666667
3,A051052,朋子,ヨガ;音楽;読書;映画鑑賞;ピクニック;料理,0.666667
4,A049050,知美,料理;音楽;ゴルフ;アート;映画鑑賞;読書,0.666667
...,...,...,...,...
76,A017018,ユウスケ,ダンス;旅行;音楽;ゴルフ;アウトドア;釣り,0.166667
77,A006007,中獅童,釣り;アウトドア;旅行;音楽;映画鑑賞;食べ歩き,0.166667
78,A010011,綾野剛志,旅行;読書;サッカー;テニス;映画鑑賞;博物館巡り,0.166667
79,A069070,亜美,ゴルフ;ダンス;音楽;バスケットボール;サッカー;フラワーアレンジメント,0.166667
